In [ ]:
import torch
import theseus as th

torch.set_default_dtype(torch.float32)

In [ ]:
def pose_error(optim_vars, aux_vars):
    pass

In [ ]:
gt_poses = [
    th.SE2(torch.tensor([[0.,0,0]]),name='gt_start'),
    th.SE2(torch.tensor([[1.,0,0]]),name='gt_finish')
]
gt_poses

In [ ]:
for g in gt_poses:
    print(g)

In [ ]:
pred_poses =[th.rand_se2(1), th.rand_se2(1)]
# pred_poses = gt_poses.copy()
pred_poses[0].name = "SE2_1"
pred_poses[1].name = "SE2_2"
pred_poses 

In [ ]:
measurement = gt_poses[0].between(gt_poses[1])
measurement

In [ ]:
measurement.name = "Odo_1"
measurement

In [ ]:
odo_w_1 = th.ScaleCostWeight(1.0, name="odo_weight_1")
odo_w_1.name

In [ ]:
odo_cost_f = th.Between(pred_poses[0], pred_poses[1], measurement, cost_weight=odo_w_1, name='odo_cf')

In [ ]:
odo_cost_f.error()

In [ ]:
objective = th.Objective()

In [ ]:
gps_cost_f = th.Difference(pred_poses[0],gt_poses[0], th.ScaleCostWeight(1.0, name="gps_weight_1"),name="gps_cf")

In [ ]:
# optim_vars = pred_poses[0], pred_poses[1]
# aux_vars = gt_poses[0], gt_poses[1]
# cost_fn = th.AutoDiffCostFunction(optim_vars, quad_error_fn, dim = 1,cost_weight=th.ScaleCostWeight(2.0, name='cost_weight'), aux_vars=aux_vars, name='se2 between func')

In [ ]:
# err = quad_error_fn(optim_vars, aux_vars)

In [ ]:
objective.add(odo_cost_f)
objective.add(gps_cost_f)

In [ ]:
objective.cost_functions_for_weights

In [ ]:
objective.aux_vars

In [ ]:
objective.optim_vars

In [ ]:
optimizer = th.LevenbergMarquardt(
    objective,
    th.CholeskyDenseSolver,
    max_iterations=50,
    step_size=0.3
)

In [ ]:
theseus_optim = th.TheseusLayer(optimizer)

In [ ]:
pred_poses =[th.rand_se2(1), th.rand_se2(1)]
pred_poses[0].name = "SE2_1"
pred_poses[1].name = "SE2_2"
pred_poses

theseus_inputs = {
    # "gt_start" : gt_poses[0],
    # "gt_finish" : gt_poses[1],
    "Odo_1" : measurement,
    "SE2_1" : pred_poses[0],
    "SE2_2" : pred_poses[1]
}

In [ ]:
theseus_inputs

In [ ]:
objective.update(theseus_inputs)

In [ ]:
with torch.no_grad():
    updated_inputs, info = theseus_optim.forward(
        theseus_inputs, optimizer_kwargs={"track_best_solution": True, "verbose":True})
print("Best solution:", info.best_solution)

In [ ]:
t_1 = info.best_solution['SE2_1']
t_2 = info.best_solution['SE2_2']

info.best_solution

In [ ]:
(t_1 - t_2).norm()

In [ ]:
(t_1 - gt_poses[0].tensor).norm()

In [ ]:
(t_2 - gt_poses[1].tensor).norm()

In [ ]:
random_rotation = th.rand_so3(1)

In [ ]:
random_rotation.tensor

In [ ]:
random_rotation.tensor.shape